In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

In [2]:
# Function to detect plateaus and estimate batch size based on amplitude range
def detect_batch_size(power_data, amplitude_range=(12, 12.5), threshold=0.01, min_plateau_length=50):
    # Calculate the difference between consecutive points
    diff = np.diff(power_data)

    # Detect where the difference is smaller than the threshold (flat regions)
    plateau_mask = np.abs(diff) < threshold

    # Find indices of plateau regions
    plateau_regions = np.where(plateau_mask)[0]

    # Group consecutive indices to identify continuous plateaus
    plateau_ranges = np.split(plateau_regions, np.where(np.diff(plateau_regions) != 1)[0] + 1)

    # Filter out short plateaus and those not within the amplitude range
    plateaus = [plateau for plateau in plateau_ranges if len(plateau) > min_plateau_length 
                and amplitude_range[0] <= power_data.iloc[plateau].mean() <= amplitude_range[1]]

    # The number of plateaus corresponds to the batch size
    batch_size = len(plateaus)

    return batch_size, plateaus

In [59]:
def process_power_readings(file_path1, file_path2):


    # Assuming CSV structure: 'Sample' (or similar), 'Power'
    df1 = pd.read_csv(file_path1)
    df2 = pd.read_csv(file_path2)

    df1.columns.values[0] = 'Sample'
    df2.columns.values[0] = 'Sample'
    # Extract sample numbers and power data
    x1 = df1['Sample']  # Replace 'Sample' with the actual column name if different
    power_data1 = df1['12V_power']  # Replace 'Power' with the actual column name if different

    x2 = df2['Sample']  # Replace 'Sample' with the actual column name if different
    power_data2 = df2['12V_power']  # Replace 'Power' with the actual column name if different

    # Detect batch size for graph 1
    batch_size1, plateaus1 = detect_batch_size(power_data1, amplitude_range=(11.6, 12.3), threshold=0.2, min_plateau_length=15)
    

    # Detect batch size for graph 2
    batch_size2, plateaus2 = detect_batch_size(power_data1, amplitude_range=(11.5, 12.5), threshold=0.2,min_plateau_length=15)

    # Optional: Plot the results
#     plt.figure(figsize=(12, 6))
#     plt.subplot(2, 1, 1)
#     plt.plot(df1['Sample'], power_data1, label='Power Data - Graph 1')
#     for plateau in plateaus1:
#         plt.plot(df1['Sample'].iloc[plateau], power_data1.iloc[plateau], 'r-', linewidth=2)
#     plt.title(f"Graph 1: Detected Batch Size = {batch_size1}")

#     plt.subplot(2, 1, 2)
#     plt.plot(df2['Sample'], power_data2, label='Power Data - Graph 2')
#     for plateau in plateaus2:
#         plt.plot(df2['Sample'].iloc[plateau], power_data2.iloc[plateau], 'r-', linewidth=2)
#     plt.title(f"Graph 2: Detected Batch Size = {batch_size2}")

#     plt.tight_layout()
    
    return batch_size1, batch_size2

In [60]:
bs_1x16x16 = 0
bs_4x8x8 = 0
for i in range(100):
    # Load data from CSV files into pandas dataframes
    # Replace with actual file paths if necessary
    file_path1 = f'hw_configs_power_profiles/power_readings_pmbus_1x16x16_{i}.csv'
    file_path2 = 'hw_configs_power_profiles/power_readings_pmbus_4x8x8.csv'
    batch_size1, batch_size2 = process_power_readings(file_path1, file_path2)
    if batch_size1 == 1:
        bs_1x16x16 += 1
    if batch_size2 == 4:
        bs_4x8x8 += 1


print(f"Detected batch size 1 for Graph 1: {bs_1x16x16} out of 100")
print(f"Detected batch size 4 for Graph 2: {bs_4x8x8} out of 100")
# plt.show()

Detected batch size 1 for Graph 1: 69 out of 100
Detected batch size 4 for Graph 2: 2 out of 100
